# predict

> Common operations around the core datastructures for running a sim

In [ ]:
#| default_exp predict

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from flax.struct import dataclass as flax_dataclass
import jax.numpy as jnp
from typing import Optional
from chewc.population import Population

In [ ]:
#| export
"""
GBLUP (Genomic Best Linear Unbiased Prediction) implementation for chewc library.

This module provides functions for genomic prediction using the GBLUP methodology,
which is a standard approach in genomic selection and animal breeding.
"""

from typing import Optional, Dict
import jax
import jax.numpy as jnp
from jax.numpy.linalg import solve, inv, pinv
from flax.struct import dataclass as flax_dataclass

from chewc.population import Population

@flax_dataclass(frozen=True)
class PredictionResults:
    """
    A container for the results of a genomic prediction.
    """
    ids: jnp.ndarray
    ebv: jnp.ndarray
    pev: Optional[jnp.ndarray] = None
    reliability: Optional[jnp.ndarray] = None
    fixed_effects: Optional[jnp.ndarray] = None
    h2_used: Optional[float] = None
    var_components: Optional[Dict] = None



In [ ]:
#| export

@jax.jit
def _calculate_genomic_relationship_matrix(geno_dosage: jnp.ndarray) -> jnp.ndarray:
    """Calculate the genomic relationship matrix (G-matrix)."""
    n_ind, n_markers = geno_dosage.shape
    
    # Calculate allele frequencies
    p = jnp.mean(geno_dosage, axis=0) / 2.0
    
    # Center genotypes
    P = 2 * p
    Z = geno_dosage - P
    
    # Calculate G matrix using VanRaden method 1
    denominator = 2 * jnp.sum(p * (1 - p))
    G = (Z @ Z.T) / denominator
    
    # Add small regularization to ensure positive definite
    epsilon = 1e-6
    G_reg = G + jnp.identity(n_ind) * epsilon
    
    return G_reg

@jax.jit
def _solve_mixed_model_equations(
    X: jnp.ndarray,
    Z: jnp.ndarray, 
    y: jnp.ndarray,
    G_inv: jnp.ndarray,
    alpha: float
) -> tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """Solve mixed model equations for GBLUP."""
    n_ind = G_inv.shape[0]
    
    # Left-hand side matrix
    XTX = X.T @ X
    XTZ = X.T @ Z
    ZTX = Z.T @ X
    ZTZ_plus_alpha_Ginv = Z.T @ Z + alpha * G_inv
    
    lhs_top = jnp.hstack([XTX, XTZ])
    lhs_bottom = jnp.hstack([ZTX, ZTZ_plus_alpha_Ginv])
    lhs = jnp.vstack([lhs_top, lhs_bottom])
    
    # Right-hand side
    XTy = X.T @ y
    ZTy = Z.T @ y
    rhs = jnp.concatenate([XTy.flatten(), ZTy.flatten()])
    
    # Solve system
    solutions = jnp.linalg.solve(lhs, rhs)
    
    # Extract solutions
    n_fixed = X.shape[1]
    fixed_effects = solutions[:n_fixed]
    random_effects = solutions[n_fixed:]
    
    # Calculate coefficient matrix inverse for PEV
    C_inv = jnp.linalg.inv(lhs)
    
    return fixed_effects, random_effects, C_inv

def gblup_predict(
    pop: Population, 
    h2: float = 0.5, 
    trait_idx: int = 0
) -> PredictionResults:
    """
    Perform GBLUP prediction with PEV calculation.
    
    Args:
        pop: Population object
        h2: Heritability
        trait_idx: Index of trait to predict
        
    Returns:
        PredictionResults object with EBVs, PEV, and reliability
    """
    # Get dosage matrix and phenotypes
    dosage = pop.dosage  # Shape: (nInd, nLoci)
    pheno = pop.pheno[:, trait_idx:trait_idx+1]  # Shape: (nInd, 1)
    
    # Remove individuals with missing phenotypes
    valid_mask = ~jnp.isnan(pheno.flatten())
    
    if jnp.sum(valid_mask) == 0:
        raise ValueError("No valid phenotypes found")
    
    dosage_valid = dosage[valid_mask]
    pheno_valid = pheno[valid_mask]
    valid_ids = pop.id[valid_mask]
    
    n_valid = jnp.sum(valid_mask)
    
    # Calculate genomic relationship matrix
    G = _calculate_genomic_relationship_matrix(dosage_valid)
    G_inv = jnp.linalg.inv(G)
    
    # Design matrices
    X = jnp.ones((n_valid, 1))  # Intercept only
    Z = jnp.identity(n_valid)   # Each individual is its own random effect
    
    # Variance ratio
    alpha = (1.0 - h2) / h2
    
    # Solve mixed model equations
    fixed_eff, random_eff, C_inv = _solve_mixed_model_equations(
        X, Z, pheno_valid, G_inv, alpha
    )
    
    # Calculate PEV and reliability
    var_p = jnp.var(pheno_valid)
    var_a = var_p * h2
    var_e = var_p * (1 - h2)
    
    # PEV from the coefficient matrix inverse (random effects part)
    n_fixed = X.shape[1]
    C22_inv = C_inv[n_fixed:, n_fixed:]
    pev = jnp.diag(C22_inv) * var_e
    
    # Reliability
    reliability = jnp.maximum(0, 1 - (pev / var_a))
    
    # Prepare results for all individuals (fill missing with zeros)
    all_ebv = jnp.zeros((pop.nInd, 1))
    all_pev = jnp.full((pop.nInd,), jnp.nan)
    all_reliability = jnp.full((pop.nInd,), jnp.nan)
    
    all_ebv = all_ebv.at[valid_mask, 0].set(random_eff)
    all_pev = all_pev.at[valid_mask].set(pev)
    all_reliability = all_reliability.at[valid_mask].set(reliability)
    
    return PredictionResults(
        ids=pop.id,
        ebv=all_ebv,
        pev=all_pev,
        reliability=all_reliability,
        fixed_effects=fixed_eff,
        h2_used=h2
    )


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()